# Chapter 8: Data Wrangling: Join, Combine, and Reshap

## 8.1: Hierarchical Indexing

In [1]:
import pandas as pd
import numpy as np

In [2]:
data=pd.Series(np.random.uniform(size=9),
               index=[["a","a","a","b","b","c","c","d","d"],[1,2,3,1,3,1,2,2,3]])
data

a  1    0.933007
   2    0.801816
   3    0.426160
b  1    0.594952
   3    0.525744
c  1    0.455902
   2    0.725246
d  2    0.256254
   3    0.805959
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data["b"]

1    0.594952
3    0.525744
dtype: float64

In [5]:
data["b":"c"]

b  1    0.594952
   3    0.525744
c  1    0.455902
   2    0.725246
dtype: float64

In [6]:
data.loc[["b","d"]]

b  1    0.594952
   3    0.525744
d  2    0.256254
   3    0.805959
dtype: float64

In [7]:
data.loc[:,2]

a    0.801816
c    0.725246
d    0.256254
dtype: float64

In [8]:
data.unstack()

,1,2,3
a,0.933007,0.801816,0.426160
b,0.594952,NaN,0.525744
c,0.455902,0.725246,NaN
d,NaN,0.256254,0.805959


In [9]:
data.unstack().stack()

a  1    0.933007
   2    0.801816
   3    0.426160
b  1    0.594952
   3    0.525744
c  1    0.455902
   2    0.725246
d  2    0.256254
   3    0.805959
dtype: float64

In [10]:
frame=pd.DataFrame(np.arange(12).reshape((4,3)),
                   index=[["a","a","b","b"],[1,2,1,2]],
                   columns=[["Ohio","Ohio","Colorado"],["Green","Red","Green"]])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [11]:
frame.index.names=["key1","key2"]
frame.columns.names=["state","color"]
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [12]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['key1', 'key2'])

In [13]:
frame.columns

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [14]:
frame.index.nlevels

2

In [15]:
frame["Ohio"]

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [16]:
pd.MultiIndex.from_arrays([["Ohio","Ohio","Colorado"],["Green","Red","Green"]],
                          names=["state","color"])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

In [17]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [18]:
frame.swaplevel("key1","key2")

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [19]:
frame.sort_index()

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [20]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [21]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [22]:
frame.columns

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

In [23]:
frame.index

MultiIndex([('a', 1),
            ('a', 2),
            ('b', 1),
            ('b', 2)],
           names=['key1', 'key2'])

### Summary Statistics by Level

In [24]:
frame.groupby(level="key2").sum()

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [25]:
frame.groupby(level="color",axis="columns").sum()

C:\Users\Karim.Badr\AppData\Local\Temp\ipykernel_15508\3295922660.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  frame.groupby(level="color",axis="columns").sum()


color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's columns

In [26]:
frame=pd.DataFrame({"a":range(7),"b":range(7,0,-1),
                    "c":["one","one","one","two","two","two","two"],
                    "d":[0,1,2,0,1,2,3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [27]:
frame2=frame.set_index(["c","d"])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [28]:
frame.set_index(["c","d"],drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [29]:
frame.set_index(["d","c"])

,,a,b
d,c,,
0,one,0,7
1,one,1,6
2,one,2,5
0,two,3,4
1,two,4,3
2,two,5,2
3,two,6,1


In [30]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [31]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 8.2: Combining and Merging Datasets

### Database-Style DataFrame Joins

In [32]:
df1=pd.DataFrame({"key":["b","b","a","c","a","a","b"],
                  "data1":pd.Series(range(7),dtype="Int64")})
df2=pd.DataFrame({"key":["a","b","d"],
                  "data2":pd.Series(range(3),dtype="Int64")})
df1
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


In [33]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [34]:
pd.merge(df1,df2,on="key")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,a,4,0
4,a,5,0
5,b,6,1


In [35]:
df3=pd.DataFrame({"lkey":["b","b","a","c","a","a","b"],
                  "data1":pd.Series(range(7),dtype="Int64")})
df4=pd.DataFrame({"rkey":["a","b","d"],
                  "data2":pd.Series(range(3),dtype="Int64")})
df3
df4

,lkey,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,rkey,data2
0,a,0
1,b,1
2,d,2


In [36]:
pd.merge(df3,df4,left_on="lkey",right_on="rkey")

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,a,2,a,0
3,a,4,a,0
4,a,5,a,0
5,b,6,b,1


In [37]:
df1
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


,key,data2
0,a,0
1,b,1
2,d,2


In [38]:
pd.merge(df1,df2,how="outer")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,c,3,<NA>
7,d,<NA>,2


In [39]:
pd.merge(df1,df2,how="right")

,key,data1,data2
0,a,2,0
1,a,4,0
2,a,5,0
3,b,0,1
4,b,1,1
5,b,6,1
6,d,<NA>,2


In [40]:
pd.merge(df1,df2,how="left")

,key,data1,data2
0,b,0,1
1,b,1,1
2,a,2,0
3,c,3,<NA>
4,a,4,0
5,a,5,0
6,b,6,1


In [41]:
pd.merge(df3, df4, left_on="lkey", right_on="rkey", how="outer")

,lkey,data1,rkey,data2
0,a,2,a,0
1,a,4,a,0
2,a,5,a,0
3,b,0,b,1
4,b,1,b,1
5,b,6,b,1
6,c,3,NaN,<NA>
7,NaN,<NA>,d,2


In [42]:
df1=pd.DataFrame({"key":["b","b","a","c","a","b"],
                  "data1":pd.Series(range(6),dtype="Int64")})
df2=pd.DataFrame({"key":["a","b","a","b","d"],
                  "data2":pd.Series(range(5),dtype="Int64")})
df1
df2

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [43]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [44]:
pd.merge(df1,df2,on="key",how="left")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,c,3,<NA>
7,a,4,0
8,a,4,2
9,b,5,1


In [45]:
pd.merge(df1,df2)

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [46]:
pd.merge(df1,df2,how="inner")

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,a,2,0
5,a,2,2
6,a,4,0
7,a,4,2
8,b,5,1
9,b,5,3


In [47]:
left=pd.DataFrame({"key1":["foo","foo","bar"],
       "key2":["one","two","one"],
       "lval":pd.Series([1,2,3],dtype="Int64")})
right=pd.DataFrame({"key1":["foo","foo","bar","bar"],
        "key2":["one","one","one","two"],
        "rval":pd.Series([4,5,6,7],dtype="Int64")})
left
right

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [48]:
pd.merge(left,right,on=["key1","key2"],how="outer")

,key1,key2,lval,rval
0,bar,one,3,6
1,bar,two,<NA>,7
2,foo,one,1,4
3,foo,one,1,5
4,foo,two,2,<NA>


In [49]:
pd.merge(left,right,on="key1")

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [50]:
pd.merge(left,right,on="key1",suffixes=("_left","_right"))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Merging on Index

In [51]:
left1 = pd.DataFrame({"key": ["a", "b", "a", "a", "b", "c"],
                      "value": pd.Series(range(6), dtype="Int64")})
right1=pd.DataFrame({"group_val":[3.5,7]},index=["a","b"])
left1
right1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


,group_val
a,3.5
b,7.0


In [52]:
pd.merge(left1,right1,left_on="key",right_index=True)

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0


In [53]:
pd.merge(left1,right1,left_on="key",right_index=True,how="outer")

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [54]:
lefth=pd.DataFrame({"key1":["Ohio","Ohio","Ohio","Nevada","Nevada"],
                    "key2":[2000,2001,2002,2001,2002],
                    "data":pd.Series(range(5),dtype="Int64")})
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [55]:
righth_index=pd.MultiIndex.from_arrays(
    [
        ["Nevada","Nevada","Ohio","Ohio","Ohio","Ohio"],
        [2001,2000,2000,2000,2001,2002]
    ]
)
righth_index

MultiIndex([('Nevada', 2001),
            ('Nevada', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2000),
            (  'Ohio', 2001),
            (  'Ohio', 2002)],
           )

In [56]:
righth=pd.DataFrame({"event1":pd.Series([0,2,4,6,8,10],dtype="Int64",index=righth_index),
                     "event2":pd.Series([1,3,5,7,9,11],dtype="Int64",index=righth_index)})

righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [57]:
lefth

,key1,key2,data
0,Ohio,2000,0
1,Ohio,2001,1
2,Ohio,2002,2
3,Nevada,2001,3
4,Nevada,2002,4


In [58]:
pd.merge(lefth,righth,left_on=["key1","key2"],right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11
3,Nevada,2001,3,0,1


In [59]:
pd.merge(lefth,righth,left_on=["key1","key2"],right_index=True,how="outer")

,key1,key2,data,event1,event2
4,Nevada,2000,<NA>,2,3
3,Nevada,2001,3,0,1
4,Nevada,2002,4,<NA>,<NA>
0,Ohio,2000,0,4,5
0,Ohio,2000,0,6,7
1,Ohio,2001,1,8,9
2,Ohio,2002,2,10,11


In [60]:
left2=pd.DataFrame([[1,2],[3,4],[5,6]],
                   index=["a","c","e"],
                   columns=["Ohio","Nevada"])
left2

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


In [61]:
right2=pd.DataFrame([[7,8],[9,10],[11,12],[13,14]],
                    index=["b","c","d","e"],
                    columns=["Missouri","Alabama"])
right2

,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


In [62]:
pd.merge(left2,right2,left_index=True,right_index=True,how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [63]:
left2.join(right2,how="outer")

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In [64]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [65]:
right1

,group_val
a,3.5
b,7.0


In [66]:
left1.join(right1,on="key")

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [67]:
left2
right2
another=pd.DataFrame([[7,8],[9,10],[11,12],[16,17]],
                     index=["a","c","e","f"],
                     columns=["New York","Oregon"])
another

,Ohio,Nevada
a,1,2
c,3,4
e,5,6


,Missouri,Alabama
b,7,8
c,9,10
d,11,12
e,13,14


,New York,Oregon
a,7,8
c,9,10
e,11,12
f,16,17


In [68]:
left2.join([right2,another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [69]:
left2.join([right2,another],how="outer")

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


In [70]:
a=pd.DataFrame({"ID":[1,2,3,4],
                "Score1":[1,1,0,1],
                "Score2":[1,1,1,1]})
a

,ID,Score1,Score2
0,1,1,1
1,2,1,1
2,3,0,1
3,4,1,1


In [71]:
b=pd.DataFrame({"ID1":[1,3,4,6,2],
                "Score3":[0,1,0,1,0],
                "Score4":[1,1,0,1,0]})
b

,ID1,Score3,Score4
0,1,0,1
1,3,1,1
2,4,0,0
3,6,1,1
4,2,0,0


In [72]:
pd.merge(a,b,left_on="ID",right_on="ID1",how="outer")

,ID,Score1,Score2,ID1,Score3,Score4
0,1.0,1.0,1.0,1,0,1
1,2.0,1.0,1.0,2,0,0
2,3.0,0.0,1.0,3,1,1
3,4.0,1.0,1.0,4,0,0
4,NaN,NaN,NaN,6,1,1


### Concatenating Along Axis

In [73]:
arr=np.arange(12).reshape((3,4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [74]:
np.concatenate([arr,arr],axis=0)

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11],
       [ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [75]:
np.concatenate([arr,arr],axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [76]:
s1=pd.Series([0,1],index=["a","b"],dtype="Int64")
s2=pd.Series([2,3,4],index=["c","d","e"],dtype="Int64")
s3=pd.Series([5,6],index=["f","g"],dtype="Int64")
s1
s2
s3

a    0
b    1
dtype: Int64

c    2
d    3
e    4
dtype: Int64

f    5
g    6
dtype: Int64

In [77]:
np.concatenate([s1,s2,s3])

array([0, 1, 2, 3, 4, 5, 6], dtype=int64)

In [78]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: Int64

In [79]:
pd.concat([s1,s2,s3],axis="columns")

,0,1,2
a,0,<NA>,<NA>
b,1,<NA>,<NA>
c,<NA>,2,<NA>
d,<NA>,3,<NA>
e,<NA>,4,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [80]:
s4=pd.concat([s1,s3])
s4

a    0
b    1
f    5
g    6
dtype: Int64

In [81]:
s1

a    0
b    1
dtype: Int64

In [82]:
pd.concat([s1,s4],axis="columns")

,0,1
a,0,0
b,1,1
f,<NA>,5
g,<NA>,6


In [83]:
pd.concat([s1,s4],axis="columns",join="inner")

,0,1
a,0,0
b,1,1


In [84]:
result=pd.concat([s1,s1,s3],keys=["one","two","three"])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: Int64

In [85]:
result.unstack()

,a,b,f,g
one,0,1,<NA>,<NA>
two,0,1,<NA>,<NA>
three,<NA>,<NA>,5,6


In [86]:
pd.concat([s1,s1,s3],axis="columns",keys=["one","two","three"])

,one,two,three
a,0,0,<NA>
b,1,1,<NA>
f,<NA>,<NA>,5
g,<NA>,<NA>,6


In [87]:
df1=pd.DataFrame(np.arange(6).reshape((3,2)),index=["a","b","c"],columns=["one","two"])
df2=pd.DataFrame(5+np.arange(4).reshape((2,2)),index=["a","c"],columns=["three","four"])
df1
df2

,one,two
a,0,1
b,2,3
c,4,5


,three,four
a,5,6
c,7,8


In [88]:
pd.concat([df1,df2],axis="columns")

,one,two,three,four
a,0,1,5.0,6.0
b,2,3,NaN,NaN
c,4,5,7.0,8.0


In [89]:
pd.concat([df1,df2],axis="columns",keys=["level1","level2"])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [90]:
pd.concat({"level1":df1,"level2":df2},axis="columns")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [91]:
pd.concat([df1,df2],axis="columns",keys=["level1","level2"],names=["upper","lower"])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [92]:
df1=pd.DataFrame(np.random.standard_normal((3,4)),columns=["a","b","c","d"])
df2=pd.DataFrame(np.random.standard_exponential((2,3)),columns=["b","d","a"])

In [93]:
df1
df2

,a,b,c,d
0,-0.919652,-0.456639,-0.965743,0.478312
1,-1.441454,-1.896476,-0.871294,0.104166
2,-0.927125,-0.185823,-1.342284,1.476132


,b,d,a
0,0.436526,0.518063,0.352337
1,0.969308,0.144658,0.114850


In [94]:
pd.concat([df1,df2])

,a,b,c,d
0,-0.919652,-0.456639,-0.965743,0.478312
1,-1.441454,-1.896476,-0.871294,0.104166
2,-0.927125,-0.185823,-1.342284,1.476132
0,0.352337,0.436526,NaN,0.518063
1,0.114850,0.969308,NaN,0.144658


In [95]:
pd.concat([df1,df2],ignore_index=True)

,a,b,c,d
0,-0.919652,-0.456639,-0.965743,0.478312
1,-1.441454,-1.896476,-0.871294,0.104166
2,-0.927125,-0.185823,-1.342284,1.476132
3,0.352337,0.436526,NaN,0.518063
4,0.114850,0.969308,NaN,0.144658


### Combining Data with Overlap

In [96]:
a=pd.Series([np.nan,2.5,0,3.5,4.5,np.nan],
            index=["f","e","d","c","b","a"])
b=pd.Series([0,np.nan,2,np.nan,np.nan,5],
            index=["a","b","c","d","e","f"])
a
b

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [97]:
pd.isna(a)

f     True
e    False
d    False
c    False
b    False
a     True
dtype: bool

In [98]:
np.where(pd.isna(a),b,a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

In [99]:
a.combine_first(b)

a    0.0
b    4.5
c    3.5
d    0.0
e    2.5
f    5.0
dtype: float64

In [100]:
df1=pd.DataFrame({"a":[1,np.nan,5,np.nan],
                  "b":[np.nan,2,np.nan,6],
                  "c":range(2,18,4)})

df2=pd.DataFrame({"a":[5,4,np.nan,3,7],
                  "b":[np.nan,3,4,6,8]})
df1
df2


,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [101]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 8.3: Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

In [102]:
data=pd.DataFrame(np.arange(6).reshape((2,3)),
                  index=pd.Index(["Ohio","Colorado"],name="state"),
                  columns=pd.Index(["one","two","three"],name="number"))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [103]:
result=data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [104]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [105]:
result.unstack(level=1)

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [106]:
result.unstack(level=0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [107]:
result.unstack(level="state")

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [108]:
s1=pd.Series([0,1,2,3],index=["a","b","c","d"],dtype="Int64")
s2=pd.Series([4, 5, 6], index=["c", "d", "e"], dtype="Int64")
s1
s2

a    0
b    1
c    2
d    3
dtype: Int64

c    4
d    5
e    6
dtype: Int64

In [109]:
data2=pd.concat([s1,s2],keys=["one","two"])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [110]:
data2.unstack()

,a,b,c,d,e
one,0,1,2,3,<NA>
two,<NA>,<NA>,4,5,6


In [111]:
data2.unstack().stack()

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: Int64

In [112]:
data2.unstack().stack(dropna=False)

C:\Users\Karim.Badr\AppData\Local\Temp\ipykernel_15508\3936770077.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data2.unstack().stack(dropna=False)


one  a       0
     b       1
     c       2
     d       3
     e    <NA>
two  a    <NA>
     b    <NA>
     c       4
     d       5
     e       6
dtype: Int64

In [113]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [114]:
pd.DataFrame({"left":result,"right":result+5},columns=pd.Index(["left","right"],name="side"))
pd.DataFrame({"left":result,"right":result+5})

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [115]:
pd.DataFrame({"left":result,"right":result+5},columns=pd.Index(["left","right"],name="side")).columns

Index(['left', 'right'], dtype='object', name='side')

In [116]:
pd.DataFrame({"left":result,"right":result+5}).columns

Index(['left', 'right'], dtype='object')

In [117]:
df=pd.DataFrame({"left":result,"right":result+5},columns=pd.Index(["left","right"],name="side"))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [118]:
df.unstack()

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [119]:
df.unstack(level="state")

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [120]:
df.unstack(level=0)

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [121]:
df.unstack(level="number")

side     left           right          
number    one two three   one two three
state                                  
Ohio        0   1     2     5   6     7
Colorado    3   4     5     8   9    10

In [122]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [124]:
df.unstack(level="state").stack(level="side")

C:\Users\Karim.Badr\AppData\Local\Temp\ipykernel_15508\2617337668.py:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df.unstack(level="state").stack(level="side")


state         Ohio  Colorado
number side                 
one    left      0         3
       right     5         8
two    left      1         4
       right     6         9
three  left      2         5
       right     7        10

Stacking converts columns to rows.
Unstacking converts the innermost level of the row index to columns.

### Pivoting "Long" to "Wide" Format

In [148]:
data=pd.read_csv("../../examples/macrodata.csv")
data

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959,1,2710.349,1707.4,286.898,470.045,1886.9,28.980,139.7,2.82,5.8,177.146,0.00,0.00
1,1959,2,2778.801,1733.7,310.859,481.301,1919.7,29.150,141.7,3.08,5.1,177.830,2.34,0.74
2,1959,3,2775.488,1751.8,289.226,491.260,1916.4,29.350,140.5,3.82,5.3,178.657,2.74,1.09
3,1959,4,2785.204,1753.7,299.356,484.052,1931.3,29.370,140.0,4.33,5.6,179.386,0.27,4.06
4,1960,1,2847.699,1770.5,331.722,462.199,1955.5,29.540,139.6,3.50,5.2,180.007,2.31,1.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,2008,3,13324.600,9267.7,1990.693,991.551,9838.3,216.889,1474.7,1.17,6.0,305.270,-3.16,4.33
199,2008,4,13141.920,9195.3,1857.661,1007.273,9920.4,212.174,1576.5,0.12,6.9,305.952,-8.79,8.91
200,2009,1,12925.410,9209.2,1558.494,996.287,9926.4,212.671,1592.8,0.22,8.1,306.547,0.94,-0.71
201,2009,2,12901.504,9189.0,1456.678,1023.528,10077.5,214.469,1653.6,0.18,9.2,307.226,3.37,-3.19


In [149]:
data=data.loc[:,["year","quarter","realgdp","infl","unemp"]]
data

,year,quarter,realgdp,infl,unemp
0,1959,1,2710.349,0.00,5.8
1,1959,2,2778.801,2.34,5.1
2,1959,3,2775.488,2.74,5.3
3,1959,4,2785.204,0.27,5.6
4,1960,1,2847.699,2.31,5.2
...,...,...,...,...,...
198,2008,3,13324.600,-3.16,6.0
199,2008,4,13141.920,-8.79,6.9
200,2009,1,12925.410,0.94,8.1
201,2009,2,12901.504,3.37,9.2


In [150]:
periods=pd.PeriodIndex(year=data.pop("year"),
               quarter=data.pop("quarter"),
               name="date")
periods

C:\Users\Karim.Badr\AppData\Local\Temp\ipykernel_15508\2873768981.py:1: FutureWarning: Constructing PeriodIndex from fields is deprecated. Use PeriodIndex.from_fields instead.
  periods=pd.PeriodIndex(year=data.pop("year"),


PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1', '1960Q2',
             '1960Q3', '1960Q4', '1961Q1', '1961Q2',
             ...
             '2007Q2', '2007Q3', '2007Q4', '2008Q1', '2008Q2', '2008Q3',
             '2008Q4', '2009Q1', '2009Q2', '2009Q3'],
            dtype='period[Q-DEC]', name='date', length=203)

In [151]:
data.index=periods.to_timestamp("D")
data

,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2
...,...,...,...
2008-07-01,13324.600,-3.16,6.0
2008-10-01,13141.920,-8.79,6.9
2009-01-01,12925.410,0.94,8.1


In [160]:
data=data.reindex(columns=["realgdp","infl","unemp"])
data.columns.name="item"
data

item,realgdp,infl,unemp
date,,,
1959-01-01,2710.349,0.00,5.8
1959-04-01,2778.801,2.34,5.1
1959-07-01,2775.488,2.74,5.3
1959-10-01,2785.204,0.27,5.6
1960-01-01,2847.699,2.31,5.2
...,...,...,...
2008-07-01,13324.600,-3.16,6.0
2008-10-01,13141.920,-8.79,6.9
2009-01-01,12925.410,0.94,8.1


In [161]:
long_data=data.stack().reset_index().rename(columns={0:"value"})
long_data

,date,item,value
0,1959-01-01,realgdp,2710.349
1,1959-01-01,infl,0.000
2,1959-01-01,unemp,5.800
3,1959-04-01,realgdp,2778.801
4,1959-04-01,infl,2.340
...,...,...,...
604,2009-04-01,infl,3.370
605,2009-04-01,unemp,9.200
606,2009-07-01,realgdp,12990.341
607,2009-07-01,infl,3.560


In [165]:
long_data.pivot(index="date",columns="item",values="value")

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


In [166]:
long_data["value2"]=np.random.standard_normal(len(long_data))
long_data

,date,item,value,value2
0,1959-01-01,realgdp,2710.349,1.451225
1,1959-01-01,infl,0.000,-0.980459
2,1959-01-01,unemp,5.800,-0.894808
3,1959-04-01,realgdp,2778.801,-0.173818
4,1959-04-01,infl,2.340,-1.411625
...,...,...,...,...
604,2009-04-01,infl,3.370,-0.232192
605,2009-04-01,unemp,9.200,0.378847
606,2009-07-01,realgdp,12990.341,0.296692
607,2009-07-01,infl,3.560,0.061473


In [168]:
pivoted=long_data.pivot(index="date",columns="item")
pivoted

value                     value2                    
item        infl    realgdp unemp      infl   realgdp     unemp
date                                                           
1959-01-01  0.00   2710.349   5.8 -0.980459  1.451225 -0.894808
1959-04-01  2.34   2778.801   5.1 -1.411625 -0.173818  1.303746
1959-07-01  2.74   2775.488   5.3 -0.507595 -0.506453 -0.459355
1959-10-01  0.27   2785.204   5.6  0.292069 -0.311736 -0.536526
1960-01-01  2.31   2847.699   5.2 -0.669180 -1.210923  0.638868
...          ...        ...   ...       ...       ...       ...
2008-07-01 -3.16  13324.600   6.0 -0.207976 -0.967163 -0.234457
2008-10-01 -8.79  13141.920   6.9  1.012901 -2.483964  1.936921
2009-01-01  0.94  12925.410   8.1  0.375664  0.784190  0.816830
2009-04-01  3.37  12901.504   9.2 -0.232192 -0.657997  0.378847
2009-07-01  3.56  12990.341   9.6  0.061473  0.296692 -3.050378

[203 rows x 6 columns]

In [169]:
pivoted["value"]

item,infl,realgdp,unemp
date,,,
1959-01-01,0.00,2710.349,5.8
1959-04-01,2.34,2778.801,5.1
1959-07-01,2.74,2775.488,5.3
1959-10-01,0.27,2785.204,5.6
1960-01-01,2.31,2847.699,5.2
...,...,...,...
2008-07-01,-3.16,13324.600,6.0
2008-10-01,-8.79,13141.920,6.9
2009-01-01,0.94,12925.410,8.1


In [173]:
unstacked=long_data.set_index(["date","item"]).unstack(level="item")
unstacked

value                     value2                    
item        infl    realgdp unemp      infl   realgdp     unemp
date                                                           
1959-01-01  0.00   2710.349   5.8 -0.980459  1.451225 -0.894808
1959-04-01  2.34   2778.801   5.1 -1.411625 -0.173818  1.303746
1959-07-01  2.74   2775.488   5.3 -0.507595 -0.506453 -0.459355
1959-10-01  0.27   2785.204   5.6  0.292069 -0.311736 -0.536526
1960-01-01  2.31   2847.699   5.2 -0.669180 -1.210923  0.638868
...          ...        ...   ...       ...       ...       ...
2008-07-01 -3.16  13324.600   6.0 -0.207976 -0.967163 -0.234457
2008-10-01 -8.79  13141.920   6.9  1.012901 -2.483964  1.936921
2009-01-01  0.94  12925.410   8.1  0.375664  0.784190  0.816830
2009-04-01  3.37  12901.504   9.2 -0.232192 -0.657997  0.378847
2009-07-01  3.56  12990.341   9.6  0.061473  0.296692 -3.050378

[203 rows x 6 columns]

Pivoting long to wide: use pivot
which is equivalent to creating a hierarchical index using set_index followed by a call to unstack

### Pivoting "Wide" to "Long" Format

In [175]:
df=pd.DataFrame({"key":["foo","bar","baz"],
                 "A":[1,2,3],
                 "B":[4,5,6],
                 "C":[7,8,9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [177]:
melted=pd.melt(df,id_vars="key",var_name="var1")
melted

,key,var1,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [178]:
reshaped=melted.pivot(index="key",columns="var1",values="value")
reshaped

var1,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


In [179]:
reshaped.reset_index()

var1,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [180]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [181]:
pd.melt(df,id_vars="key",value_vars=["A","B"])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [182]:
pd.melt(df,value_vars=["A","B","C"])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [184]:
pd.melt(df,value_vars=["key","A","B"])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
